# 구글 플레이 스토어 리뷰 웹 크롤러
- Contributor: Tony Park
- References
    - Tistory Blog | https://heytech.tistory.com/
    - Github | https://github.com/park-gb
- Last Updated@2022-01-27

# 초기 설정

## 크롬 드라이버 세팅

In [1]:
# chrome_driver = '../chromedriver.exe' # 파일 확장자 이름 표기
chrome_driver = 'chromedriver.exe' # 파일 확장자 이름 미표기

## 수집 앱 URL

In [6]:
URL = 'https://play.google.com/store/apps/details?id=com.kakaobank.channel&hl=ko&gl=US'

# 패키지 import

In [7]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep
import random
from tqdm.auto import tqdm, trange
import pandas as pd

# 무한 스크롤 함수

In [8]:
def scroll(modal):
    try:        
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
        while True:
            pause_time = random.uniform(0.5, 0.8)
            # 최하단까지 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height
                
    except Exception as e:
        print("에러 발생: ", e)

# 데이터 크롤링

In [9]:
# 크롬 드라이버 세팅
driver = webdriver.Chrome(chrome_driver)
# 페이지 열기
driver.get(URL)
# 페이지 로딩 대기
wait = WebDriverWait(driver, 5)

# '리뷰 모두 보기' 버튼 렌더링 확인
all_review_button_xpath = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/div/div/div[5]/div/div/button/span'
button_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_button_xpath)))
# '리뷰 모두 보기' 버튼 클릭
driver.find_element_by_xpath(all_review_button_xpath).click()

# '리뷰 모두 보기' 페이지 렌더링 대기
all_review_page_xpath = '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div'
page_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_page_xpath)))

# 페이지 무한 스크롤 다운
modal = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
scroll(modal)

C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_18444/1607492057.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver)
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_18444/1607492057.py:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(all_review_button_xpath).click()
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_18444/800829709.py:18: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()


에러 발생:  Message: stale element reference: element is not attached to the page document
  (Session info: chrome=102.0.5005.63)
Stacktrace:
Backtrace:
	Ordinal0 [0x00E0D953+2414931]
	Ordinal0 [0x00D9F5E1+1963489]
	Ordinal0 [0x00C8C6B8+837304]
	Ordinal0 [0x00C8F0B4+848052]
	Ordinal0 [0x00C8EF72+847730]
	Ordinal0 [0x00C8F8EA+850154]
	Ordinal0 [0x00CE5249+1200713]
	Ordinal0 [0x00CD449C+1131676]
	Ordinal0 [0x00CE4812+1198098]
	Ordinal0 [0x00CD42B6+1131190]
	Ordinal0 [0x00CAE860+976992]
	Ordinal0 [0x00CAF756+980822]
	GetHandleVerifier [0x0107CC62+2510274]
	GetHandleVerifier [0x0106F760+2455744]
	GetHandleVerifier [0x00E9EABA+551962]
	GetHandleVerifier [0x00E9D916+547446]
	Ordinal0 [0x00DA5F3B+1990459]
	Ordinal0 [0x00DAA898+2009240]
	Ordinal0 [0x00DAA985+2009477]
	Ordinal0 [0x00DB3AD1+2046673]
	BaseThreadInitThunk [0x75AAFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77A97A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77A97A4E+238]



# HTML Parsing

In [9]:
# html parsing하기
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

# HTML 데이터 저장

In [16]:
# html 데이터 저장
with open("data_html.html", "w", encoding = 'utf-8') as file:
    file.write(str(soup_source))

## 데이터프레임 변환

In [13]:
# 리뷰 데이터 클래스 접근
review_source = soup_source.find_all(class_ = 'RHo1pe')
# 리뷰 데이터 저장용 배열
dataset = []
# 데이터 넘버링을 위한 변수
review_num = 0 
# 리뷰 1개씩 접근해 정보 추출
for review in tqdm(review_source):
    review_num+=1
    # 리뷰 등록일 데이터 추출
    date_full = review.find_all(class_ = 'bp9Aid')[0].text
    date_year = date_full[0:4] # 연도 데이터 추출
    # 해당 단어가 등장한 인덱스 추출
    year_index = date_full.find('년')
    month_index = date_full.find('월')
    day_index = date_full.find('일')
    
    date_month = str(int(date_full[year_index+1:month_index])) # 월(Month) 데이터 추출
    # 월 정보가 1자리의 경우 앞에 0 붙이기(e.g., 1월 -> 01월)
    if len(date_month) == 1:
        date_month = '0' + date_month
    
    date_day = str(int(date_full[month_index+1:day_index])) # 일(Day) 데이터 추출 
    # 일 정보가 1자리의 경우 앞에 0 붙여줌(e.g., 7일 -> 07일)
    if len(date_day) == 1:
        date_day = '0' + date_day
    
    # 리뷰 등록일 full version은 최종적으로 yyyymmdd 형태로 저장
    date_full = date_year + date_month + date_day
    user_name = review.find_all(class_ = 'X5PpBb')[0].text # 닉네임 데이터 추출
    rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][10] # 평점 데이터 추출
    content = review.find_all(class_ = 'h3YV2d')[0].text # 리뷰 데이터 추출

    data = {
        "id": review_num, 
        "date": date_full,
        "dateYear": date_year,
        "dateMonth": date_month,
        "dateDay": date_day,
        "rating": rating,
        "userName": user_name,
        "content": content
    }
    dataset.append(data)

  0%|          | 0/2800 [00:00<?, ?it/s]

## 데이터프레임 저장

In [15]:
df = pd.DataFrame(dataset)
df.to_csv('review_dataset.csv', encoding = 'utf-8-sig') # csv 파일로 저장

# 리뷰 데이터 불러오기

In [18]:
# 저장한 리뷰 정보 불러오기
df = pd.read_csv('review_dataset.csv', encoding = 'utf-8-sig')
df = df.drop(['Unnamed: 0'], axis = 1) # 불필요한 칼럼 삭제
df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
0,1,20220314,2022,3,14,3,JM K,자꾸 핸드폰 인증하라고 반복되는데 그때마다 매번 인증해야해서 번거롭기도하고 너무 짜...
1,2,20220527,2022,5,27,2,진보영,카뱅 잘 사용 하는데 2일에 한번씩 들어갈일이 있으면 보안 os 업그레이드하래요. ...
2,3,20220415,2022,4,15,2,하이하이14,평소에 잘 사용해오던 앱이였는데 제가 이상한건지는 모르겠는데 최근에 다른 사람에게 ...
3,4,20220421,2022,4,21,1,SoSFXP FL,오픈뱅킹 화면이 토스처럼 직관적이질 않아서 손이 안가는군요. 첫화면부터가 카뱅계좌가...
4,5,20220504,2022,5,4,1,Taek Oh,"이미 대출승인내서 어플 다 쓰게만들어놓고서는 뒤늦게 신분증 다시 제출하라고하고, 신..."
...,...,...,...,...,...,...,...,...
2795,2796,20210903,2021,9,3,1,Hyunsik Yu,대출이자는 처음엔 싸다가 슬금슬금 올려서 딴은행보다 비싸고 예금금리는 제일 낮고 별...
2796,2797,20200711,2020,7,11,5,박라연,수수료 없고 편리한 카뱅 최고
2797,2798,20210312,2021,3,12,1,Master G,거지같은앱 신분증사진요구하고 권한사항미동의시 사용불가능 개인정보 빼가려는 듯한 사기앱같음
2798,2799,20200720,2020,7,20,5,Roy Jang,매우 편리하고 유용합니다.
